In [1]:
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D, MaxPooling2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from sklearn.utils import shuffle

import matplotlib.pyplot as plt

import sys

import numpy as np

import os

from PIL import Image

Using TensorFlow backend.


In [2]:
def load_data():
        data = []
        paths = []
        y = []
        for r, d, f in os.walk(r"D:\Downloads\cats-faces-64x64-for-generative-models\cats"):
            for file in f:
                if '.jpg' in file:
                    paths.append(os.path.join(r, file))

        #for each file add normal resolution and small resolution to arrays
        for path in paths:
            img = Image.open(path)
            #x = img.resize((64,64))
            placeholder = np.array(img)
            #placeholder = placeholder[...,:3]
            data.append(placeholder)
                
        x_train = np.load(r'C:\Users\Vee\Desktop\python\CNNs\trash\pca_dataset.npy')
        y_train = np.array(data)
        y_train = y_train.reshape(len(data),64,64,3)
        X_shuffle, Y_shuffle = shuffle(x_train, y_train)
        return X_shuffle, Y_shuffle

In [3]:
class DCGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 64
        self.img_cols = 64
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 512

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()
        generator = self.generator

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()
        model.add(Dense(8 * 8 * 8, input_dim=self.latent_dim))
        model.add(Reshape((8, 8, 8)))
        
        model.add(Conv2D(64, kernel_size=(5,5), padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        
        model.add(UpSampling2D())
        
        model.add(Conv2D(64, kernel_size=(5,5), padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        
        model.add(UpSampling2D())

        model.add(Conv2D(64, kernel_size=(5,5), padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        
        model.add(UpSampling2D())

        model.add(Conv2D(64, kernel_size=(5,5), padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        
        model.add(Conv2D(3, kernel_size=(5,5), padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(64, kernel_size=(5,5), strides=3, input_shape=self.img_shape, padding="same",activation = "relu"))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=(5,5), activation='relu', padding="same"))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=(5,5), activation='relu', padding="same"))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=(5,5), activation='relu', padding="same"))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=(5,5), activation='relu', padding="same"))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=(5,5), activation='relu', padding="same"))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)
    

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        X_train, Y_train = load_data()
        
        Y_train = Y_train/255

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):
            start = 0
            end = start + batch_size
            for i in range(int(len(X_train)/batch_size)):
                imgs = Y_train[start:end]
                noise = X_train[start:end]

                # ---------------------
                #  Train Discriminator
                # ---------------------

                # Select a random half of images
                gen_imgs = self.generator.predict(noise)

                # Train the discriminator (real classified as ones and generated as zeros)
                d_loss_real = self.discriminator.train_on_batch(imgs, valid)
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                # Train the generator (wants discriminator to mistake images as real)
                g_loss = self.combined.train_on_batch(noise, valid)
                
                start = start + batch_size
                end = end + batch_size
            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                # Select a random indexes
                idx = np.random.randint(0, X_train.shape[0], 8)
                x_points = X_train[idx]
                y_points = Y_train[idx]
                self.save_imgs(epoch, self.generator.predict(x_points), y_points)

    def save_imgs(self, epoch, gen_imgs, y_points):
        r, c = 4, 4
        gen_imgs = np.array(gen_imgs) * 255
        gen_imgs = gen_imgs.astype(int)
        y_points = np.array(y_points) * 255
        y_points = y_points.astype(int)
        
        fig=plt.figure(figsize= (40, 40))
        
        img_count = 0;
        index_count = 0;
        y_count = 0;
        
        for i in range(1, c+1):
            for j in range(1, r+1):
                if(j % 2 == 0):
                    img = y_points[y_count]
                    y_count = y_count+1
                else:
                    img = gen_imgs[index_count]
                    index_count = index_count+1
                fig.add_subplot(r, c, img_count+1)
                plt.imshow(img)
                img_count = img_count + 1
        fig.savefig(r"C:\Users\Vee\Desktop\python\GAN\pca_gan\epoch_%d.png" % epoch)
        plt.close()
        # save model to .h5 file
        self.generator.save(r"C:\Users\Vee\Desktop\python\GAN\pca_gan\generator" + str(epoch) + ".h5")

In [4]:
dcgan = DCGAN()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 22, 22, 64)        4864      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 11, 11, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 11, 11, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        102464    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 5, 5, 64)        

In [ ]:
dcgan.train(epochs=50001, batch_size=181, save_interval=10)

C:\Users\Vee\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
C:\Users\Vee\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
C:\Users\Vee\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input da

0 [D loss: 0.693163, acc.: 47.51%] [G loss: 0.693213]
1 [D loss: 0.693418, acc.: 44.75%] [G loss: 0.692735]
2 [D loss: 0.693251, acc.: 48.34%] [G loss: 0.693739]
3 [D loss: 0.693701, acc.: 44.20%] [G loss: 0.691942]
4 [D loss: 0.693148, acc.: 53.04%] [G loss: 0.693614]
5 [D loss: 0.692889, acc.: 56.63%] [G loss: 0.693245]
6 [D loss: 0.694923, acc.: 49.45%] [G loss: 0.697824]
7 [D loss: 0.693154, acc.: 51.93%] [G loss: 0.702845]
8 [D loss: 0.687361, acc.: 69.34%] [G loss: 0.710170]
9 [D loss: 0.686384, acc.: 68.51%] [G loss: 0.712705]


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


10 [D loss: 0.694803, acc.: 45.30%] [G loss: 0.705492]
11 [D loss: 0.693401, acc.: 50.55%] [G loss: 0.705847]
12 [D loss: 0.660430, acc.: 50.00%] [G loss: 0.649424]
13 [D loss: 0.587969, acc.: 69.34%] [G loss: 1.468977]
14 [D loss: 0.665234, acc.: 61.05%] [G loss: 1.308643]
15 [D loss: 0.616623, acc.: 66.57%] [G loss: 1.119421]
16 [D loss: 0.645850, acc.: 64.64%] [G loss: 1.256846]
17 [D loss: 0.674498, acc.: 59.12%] [G loss: 0.969952]
18 [D loss: 0.656501, acc.: 60.22%] [G loss: 0.958721]
19 [D loss: 0.682770, acc.: 57.73%] [G loss: 0.979712]


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


...

680 [D loss: 0.030870, acc.: 99.17%] [G loss: 8.798630]
681 [D loss: 0.028315, acc.: 98.90%] [G loss: 7.623681]
682 [D loss: 0.010144, acc.: 99.72%] [G loss: 8.321249]
683 [D loss: 0.081753, acc.: 96.41%] [G loss: 7.329447]
684 [D loss: 0.009766, acc.: 99.72%] [G loss: 8.205248]
685 [D loss: 0.059688, acc.: 98.90%] [G loss: 7.660392]
686 [D loss: 0.032642, acc.: 99.17%] [G loss: 8.832496]
687 [D loss: 0.144841, acc.: 93.92%] [G loss: 7.122180]
688 [D loss: 0.067021, acc.: 96.96%] [G loss: 7.411631]
689 [D loss: 0.045804, acc.: 98.34%] [G loss: 8.761347]


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


690 [D loss: 0.028516, acc.: 98.90%] [G loss: 6.930908]
691 [D loss: 0.042448, acc.: 98.34%] [G loss: 6.662126]
692 [D loss: 0.068294, acc.: 97.79%] [G loss: 6.457507]
693 [D loss: 0.028323, acc.: 98.90%] [G loss: 7.272249]
694 [D loss: 0.078320, acc.: 97.79%] [G loss: 6.951348]
695 [D loss: 0.038931, acc.: 98.07%] [G loss: 6.309139]
696 [D loss: 0.093883, acc.: 96.41%] [G loss: 6.319352]
697 [D loss: 0.072756, acc.: 97.51%] [G loss: 5.710264]
698 [D loss: 0.083850, acc.: 97.51%] [G loss: 5.612433]
699 [D loss: 0.101943, acc.: 96.13%] [G loss: 6.344057]


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


700 [D loss: 0.103002, acc.: 95.86%] [G loss: 5.808981]
